In [20]:
#%pylab inline
%matplotlib notebook

In [21]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import re

## Get Mask

In [22]:
diri = r'C:\Users\apbarret\Documents\data\ancillary'
maskFile = r'arctic_mask_1x1deg.nc'
maskIn = xr.open_dataset(os.path.join(diri, maskFile))
maskIn

<xarray.Dataset>
Dimensions:      (lat: 180, lon: 360)
Coordinates:
  * lat          (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 -84.5 -83.5 ...
  * lon          (lon) float32 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5 ...
Data variables:
    arctic_mask  (lat, lon) int8 ...
Attributes:
    creation_date:  Thu Apr 15 17:39:34 2010
    created_by:     Andrew P. Barrett
    Description:    Arctic land and ocean mask, and global land and ocean mask
    Source:         Original Arctic drainage and ocean mask was RIMS Budgetee...

In [24]:
maskIn['arctic_mask'].plot()

## Get grid coordinates

In [25]:
gridFile = r'ease_nh50km_coordinates.nc'
grid = xr.open_dataset(os.path.join(diri, gridFile))
grid['lat2d'] = grid['lat2d'].where(grid['lat2d'] > -999.)
grid['lon2d'] = grid['lon2d'].where(grid['lon2d'] > -999.)
grid['lon2d'] = grid['lon2d'].where(grid['lon2d'] > 0., grid['lon2d']+360.)
print (grid['lon2d'].min(), grid['lon2d'].max())

<xarray.DataArray 'lon2d' ()>
array(0.07991) <xarray.DataArray 'lon2d' ()>
array(359.760284)


In [26]:
plt.imshow(grid['lon2d'])

In [27]:
lat = grid['lat2d'].values.reshape(-1)
lon = grid['lon2d'].values.reshape(-1)
idx = np.where(np.isfinite(lat) & np.isfinite(lon))

In [34]:
newMask1D = np.empty(lat.shape)
newMask1D[:] = np.nan
newMask1D[idx] = maskIn['arctic_mask'].sel(lat=xr.DataArray(lat[idx]), 
                                              lon=xr.DataArray(lon[idx]), 
                                              method='nearest').values


In [31]:
plt.imshow(maskOutArray.reshape(grid['lat2d'].shape))

In [41]:
newMask = xr.Dataset({'arctic_mask': (['x','y'], newMask1D.reshape(grid['lat2d'].shape))},
                     coords = {'lat': (['x','y'], grid['lat2d']),
                               'lon': (['x','y'], grid['lon2d'])})
newMask

<xarray.Dataset>
Dimensions:      (x: 360, y: 360)
Coordinates:
    lat          (x, y) float32 nan nan nan nan nan nan nan nan nan nan nan ...
    lon          (x, y) float32 nan nan nan nan nan nan nan nan nan nan nan ...
Dimensions without coordinates: x, y
Data variables:
    arctic_mask  (x, y) float64 nan nan nan nan nan nan nan nan nan nan nan ...

In [ ]:
newMask['arctic_mask'].plot()